In [ ]:
#!pip install simplet5
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle/

In [ ]:
!kaggle datasets download -d nmaria/news-t5-model

In [ ]:
!unzip /content/drive/MyDrive/Kaggle/news-t5-model.zip

In [ ]:
!kaggle datasets download -d nmaria/lentaru-20202021

In [ ]:
!unzip /content/drive/MyDrive/Kaggle/lentaru-20202021.zip

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/Kaggle/lenta-ru-news.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Kaggle/lenta-ru-news_1.csv')
train_df = pd.read_csv('../input/noisy-news/train_t5.csv', header=0, names=['ind', 'input_text', 'target_text'], usecols=['input_text', 'target_text']) 
eval_df = pd.read_csv('../input/noisy-news/test_t5.csv', header=0, names=['ind', 'input_text', 'target_text'], usecols=['input_text', 'target_text']) 
df =  pd.concat([df1, df2], ignore_index=True)
df['text'] = df['text'].apply(lambda x: 'grammar: ' + x)
texts = df['text'].tolist()[-20617:]

In [ ]:
from pprint import pprint
import pandas as pd
from simpletransformers.t5 import T5Model

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 2,
    "num_train_epochs": 2,
    "max_length": 512,
    "length_penalty": 10,
    #"save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": False,
    "fp16": False,
    #"wandb_project": "GEC with T5",
}
model = T5Model('mt5', "/content/drive/MyDrive/Kaggle/outputs", use_cuda=True, use_multiprocessing=False)

#eval_df = pd.read_csv('../input/noisy-news/test_t5.csv', header=0, names=['ind', 'input_text', 'target_text'], usecols=['input_text', 'target_text']) 
#eval_df['input_text'] = eval_df['input_text'].apply(lambda x: x[9:])
#eval_df['prefix'] = ['grammar'] * len(eval_df)
model.train(train_df)

In [ ]:
from pprint import pprint
import pandas as pd
from simpletransformers.t5 import T5Model


model = T5Model('mt5', "/content/drive/MyDrive/Kaggle/outputs", use_cuda=True, use_multiprocessing=False)

preds = model.predict(texts)

pred_df = pd.DataFrame({'preds': preds, 'input': texts})
pred_df.to_csv('/content/drive/MyDrive/Kaggle/pred_t5_full.csv', index=False)